# Welcome to Prompt Parrot! 🦜

This notebook is designed to train language model on a list of your prompts,generate prompts in your style, and synthesize wonderful surreal images! ✨ 

This notebook is created and maintained by [Stephen Young](https://twitter.com/KyrickYoung) or SteveTheNinja#0616 on discord

# Models Used
- Prompt Parrot is primarily powered by `distilgpt2`, which is a finetuned [GPT-2 model by huggingface](https://huggingface.co/distilgpt2). 
- And [Stable Diffusion](https://huggingface.co/CompVis/stable-diffusion) for image generation a diffusion made by CompVis in colaboration with Stability AI.

# Changelog
###2.0
- Streamlined model training UX!
- Added ability to save multiple finetuned models! 
- Gave the parrot a paintbrush! Prompt Parrot will now generate images with Stable Diffusion!
- Added image grid options
- Provided a list of default prompts! Prompt Parrot now works out of the box! 
- Added NSFW retry setting. The parrot will try another seed until finally giving up in shame that it can't generate a SFW image.

###2.1
- Added ability to create multiple grids! More grids, more fun!
- Sneaked in extra prompts

###2.2
- Silence the image progress bar. This was a major performance bottleneck. Stable Diffusion is VERY fast

###2.3
- Implement prompt-parrot common gen functions

### 2.5
- Completely overhauled the training strategy! Generated prompts are more coherent. 
- Added ~30 more prompts!! The new total is 185 built-in prompts
- Added prompt overrides so you can select fewer prompts to run and edit them. This will override all generated prompts. To undo simply delete the overrides.
- Improved hyperparameter instructions

# Instructions

This notebook has two primary finetuning modes:
1. Train on built in prompts
1. Train on a prompt file.

### Option 1) Builtin Prompts

Run all cells! Default settings will use the builtin prompts by default, train GP2 and produce art!

### Option 2) Prompt File

1. Create a text file with all of your prompts!

    **Prompt Parrot requires a minimum of 30 prompts to train effectively.** Less will work, but the results will tend to overfit. More than 50 prompts is ideal! The more the merrier. Try to get as many prompts as you can into the text file!

    Example prompt file:
    ```
    a beautiful painting of a mountain by Tyler Edlin, trending on arstation
    a stunning painting of Chicago, oil and canvas
    cyberpunk hoover dam, cgsociety, artstation, highly detailed matte painting
    ```

2. Next upload your file to the session

    ![upload_icon.png](https://drive.google.com/uc?id=14Is1KFTqpRwqeCWuIPC3StQsAbmG6als)

3. Finally copy the path to your uploaded file and paste it below!

    ![upload_icon.png](https://drive.google.com/uc?id=1sjvjMc3mUkmbNgh3SkRd616A_Gvol5zP)

In [ ]:
!pip install transformers datasets scipy ftfy diffusers==0.9.0
!pip install git+https://github.com/kyrick/parrot-tools.git --no-deps


In [ ]:
import random
from pathlib import Path
import os

# image gen imports
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from parrot_tools.generate import run_prompts, BatchSettings, Prompt, set_scheduler

# GPT2 training imports
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TextDataset, 
    DataCollatorForLanguageModeling,
    Trainer, 
    TrainingArguments
)
import datasets
datasets.disable_caching()

# THIS IS A DIRTY HACK TO SILENCE THE PROGRESS BAR
# THE TQDM PROGRESS BAR BOTTLENECKS CELL OUTPUT AND SLOWS THE NOTEBOOK
from tqdm.auto import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

# setup paths
NOTEBOOK_PATH = Path("/notebooks")
IMAGES_OUT = NOTEBOOK_PATH / "images_out"
CACHE_PATH = NOTEBOOK_PATH / "models"

os.makedirs(IMAGES_OUT, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)


In [ ]:
# Define functions

start_token = "<BOP>"
pad_token = "<PAD>"
end_token = "<EOP>"
special_tokens_dict = {
    'bos_token': start_token, 
    'eos_token': end_token, 
    'pad_token': pad_token
}

def tokenize_line_by_line(tokenizer, prompts_dataset):
    def tokenize_function(dataset):
        # Remove empty lines

        dataset["prompts"] = [
            start_token + prompt.strip() + end_token 
            for prompt in dataset["prompts"] 
            if len(prompt.strip()) > 0]

        return tokenizer(
            dataset["prompts"],
            padding="max_length",
            truncation=True,
            # return_special_tokens_mask=True,
            max_length=tokenizer.model_max_length,
        )

    tokenized_dataset = prompts_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=10000,
        drop_last_batch=False,
        remove_columns=["prompts"],
    ).map(remove_columns=['attention_mask'])
    return tokenized_dataset


def tokenize_group(tokenizer, prompts_dataset, shuffles=4):
    def group_texts(examples):
        block_size = tokenizer.model_max_length
        
        prompts = examples["prompts"]

        shuffled_prompts = ""
        for _ in range(shuffles):
            random.shuffle(prompts)
            concat =  "".join([start_token + prompt.strip() + " " + end_token for prompt in prompts if len(prompt.strip()) > 0])
            shuffled_prompts += concat

        input_ids = tokenizer(shuffled_prompts)["input_ids"]

        result = []
        for i in range(0, len(input_ids) - block_size + 1, block_size):  # Truncate in block of block_size
            result.append(
                tokenizer.build_inputs_with_special_tokens(input_ids[i : i + block_size])
            )
        return {"input_ids":result}

    tokenized_datasets = prompts_dataset.map(
        group_texts,
        batched=True,
        batch_size=10000,
        remove_columns=["prompts"],
    )
    return tokenized_datasets



In [ ]:
import textwrap

builtin_prompts = """\
an oil tanker drydock constructed in a field in the countryside by John Constable and Tyler Edlin, artstation
ww1 warship in drydock near an European village countryside landscape by Casper David Friedrich, artstation
cossacks marching near a warship in the countryside landscape by Tyler Edlin and Caspar David Friedrich, artstation
a crashed warship, a fishing village on the river in a mountain landscape by Casper David Friedrich, warship on artstation, mountain landsape on artstation, village on artstation
a flower field landscape, an abandoned building in a field, a mural of flowers on the side of an abandoned building, a rusty car, trending on artstation, cinematic lighting, vibrant
a shipwreck on a sci-fi alien desert landscape, german romanticism, artstation, digital art, magical realism, moody palette, contrasting
shipwreck on a twisted alien landscape, abandoned twisted alien landscape by Casper David Friedrich, whispy clouds, planets and stars in the sky, artstation
a flower field landscape, an abandoned castle in a field, a mural of flowers on the side of an abandoned castle, petals flying in the breeze, butterflies, artstation, cinematic lighting, vibrant
an abandoned skyscraper in a field of flowers, petals flying in the breeze, flowers made of metal shards, cinematic lighting, vibrant
an abandoned skyscraper in a field in an autumn pastoral landscape, cinematic lighting, vibrant, surrealism, deviantart
an abandoned boat in a frozen winter wonderland lake, snowflakes drifting in the breeze, impressionism, magical realism
a sailboat docked at a boathouse, snowflakes drifting in the breeze, cinematic lighting, vibrant, winter wonderland
an abandoned building in a field of flowers landscape by caspar david friedrich and RHADS, artstation, cinematic lighting
an iridescent cottage on a crumbling cliffside by the sea, by Tyler Edlin and Andreas Rocha, deviantart, digital painting
a surreal digital art illustraiton of an abandoned castle in a field of flowers landscape by Victo Ngai, flowers, butterflies
a glowing obelisk on a mountain by Alejandro Burdisio and Tyler Edlin and RHADS, digital paintnig, award-winning
ruins of an abandoned medieval city in wasteland landscape by Felix Kelly and Richard Anderson, oil on canvas, american romanticism
an abandoned city in a tropical swamp landscape by Gediminas Pranckevicius and Jacek Yerka, murals of tropical flowers, artstation, vibrant, cinematic lighting
an abandoned city in a lakes and field of flowers landscape, oil on canvas, award-winning, textured, moody palette, deviantart
an abandoned building in a field of flowers by Jean Delville and Thomas Kinkade, a mural of flowers on an abandoned building, highly detailed matte painting, vibrant
an abandoned cathedral in a field of flowers by Jean Delville and John Atkinson Grimshaw, highly detailed matte painting, german romanticism
streets of the abandoned city overgrown with blooming vines by Jean Delville and John Atkinson Grimshaw and Gilbert Williams, concept art
lightning storm over the grand canyon, dramatic scenery, cinematic lighting, graphic illustration, by Gilbert Williams and RHADS
swirling green plasma, infinite celestial sea of darkness, deviantart, digital art, contrasting, simple subject
swarm of green fireflies over a dark nightscape landscape by Gilbert Williams and Beeple, digital painting, award winning, artstation
a dark wizard tower photographed by ansel adams and painted by RHADS, oil on canvas
an aquarium in a dimly lit room, abandoned industrial room, low hanging flourescent light, digital painting, contest-winner
an aquarium in a dimly lit abandoned industrial room, low hanging fluorescent light, artstation
imposing glowing sci-fi neon obelisks on a snowy landscape, falling snow, oil on canvas, artstation
60's mainframes in a dimly lit abandoned industrial room, low hanging fluorescent light, digital art, clean, dark, high-contrast, moody palette
60's mainframes in a dimly lit abandoned industrial room, face renders on computer screens with face renders, low hanging fluorescent light, trending on artstation
a cylinder aquarium in a dimly lit abandoned warehouse, low hanging fluorescent light, lab equipment, trending on artstation
a tropical aquarium in a dimly lit abandoned atrium, skylights, trending on artstation, steampunk
an aquarium in an atrium, a crowd gathers, skylights, trending on artstation, steampunk, victorian
a panel aquarium in a dim abandoned warehouse, low hanging fluorescent light, artstation, digital art, 35mm photograph
abandoned cyberpunk lab, an aquarium in a dimly lit cyberpunk lab, low hanging fluorescent light, by Syd Mead and James Turrell
intricate crumbling ivy covered wall by claude monet, highly detailed painting, artstation
a majestic dieselpunk battlemech standing near a russian orthodox church on a snowing landscape, by Gilbert Williams and Barclay Shaw, concept art
an imposing dieselpunk mech in the autumn countryside by Ivan Aivazovsky and Greg Rutkowski, trending on artstation, cinematic lighting
an imposing clockwork elephant mech on an arctic landscape by Ivan Aivazovsky and Greg Rutkowski, digital painting, highly detailed, award winning concept art
a space marine bear by Ivan Aivazovsky and Greg Rutkowski, warhammer 40k
a space marine bear by george stubbs and greg rutkowski, warhammer 40k, illustration
post-industrial smokestacks on lava flows across the wasteland with belching smoke filling the sky, red lighting lacing the clouds, by Ivan Aivazovsky and Greg Rutkowski, artstation
giant sinister twisted rose vines cover the wizard tower in the forest by tyler edlin and gilbert williams, highly detailed matte painting, surrealism 
an imposing lighthouse in a nebula by tyler edlin and RHADS, artstation, oil on canvas, magical realism, science fiction concept art
a beautiful painting of a circular portal opening to a flower ocean dimension embedded in an asteroid floating in a nebula by RHADS, trending on deviantart, vibrant
a beautiful painting of a tree on an asteroid in a vast nebula by Albert Bierstadt and Tyler Edlin and RHADS
a painting of an ancient magical mystical willow tree on an asteroid in a vast nebula by Tyler Edlin and RHADS and Gilbert Williams
a stunning portrait of a woman pondering the wilderness by RHADS and Gilbert Williams, artstation, digital painting
an abandoned building on an asteroid in a vast nebula by Tyler Edlin and John Harris and Hubert Robert, concept art, watercolor
a spanish galleon sailing on a vast and beautiful nebula by ivan aivazovsky and John Harris, celestial sea
a beautiful painting of a sailboat sailing on a nebula by RHADS and Gilbert Williams, deviantart
whales swimming through vast and beautiful clouds by RHADS and Ross Tran and Albert Bierdstat, artstation, god rays, surrealism
inside an abandoned 1980's mall, retrowave, 35mm photograph, Memphis Design, HD photograph, octane soft render
empty waiting room with yellow wallpaper, found photograph, by ansel adams, washed out color
empty unsettling waiting room with yellow wallpaper, 3d room render, trending on artstation
an imposing mech standing in a wheat field on an overcast summer day by john constable, artstation
a beautiful painting of an abandoned rusty cyberpunk city, neon signs, in the style of syd mead and greg rutkowski, artstation
an abandoned and decaying city screenprint by dan mumford, cuberpunk style, blinking neon signs, artstation
a illustration of a diner on the street corner of a rusty cyberpunk city, neon signs, trending on artstation
a diorama of colorful soil layers, waterfalls between layers over the wooden buildings between the layers, magic realism painting by Alexander Jansson and machinarium
on the streets of the ancient city sinking into the briny deep by Tyler Edlin, artstation, vivid, vibrant
Zoroastrian towers on a desert landscape by Andreas Rocha and Hubert Robert, digital art, magical realism, fantasy concept art
portrait of the devine cybernetic angel by Peter Mohrbacher and Ross Tran, character design, octane soft render, cyberpunk
empress of clockwork spacetime painted by Peter Mohrbacher and Ross Tran
stunning beautiful portrait of clockwork singularity princess painted by Peter Mohrbacher and WLOP, rossdraws, digital painting
portrait of a stunning beautiful explosion over the ocean by Peter Mohrbacher, character design, surrealism, vibrant palette
a luminous orchid garden painted by Albert Bierstadt and Hubert Robert, oil on canvas, magical realism, luminous
painting of a secret cottage in a luminous orchid garden by Albert Bierstadt and Jean Delville, artstation
mystical painting of a cute dark orchid garden by Alexander Jansson
stunning painting of a lone rusted imposing mech in a wheatfield by Simon Stålenhag and beeple, digital painting, soft render, moody palette
an imposing mecha-samurai hybrid in a field of violet flowers by Gediminas Pranckevicius and beeple, dieselpunk aesthetic
a landscape painting of retro diner outside los angeles by Peter Mohrbacher and Simon Stålenhag, retrowave noir
a lighthouse on a cliffside overlooking the turbulent bay by Tyler Edlin and ivan aivazovsky
cherry blossom tree next to a serene river by Ross Tran and Tyler Edlin, magical realism, impressionism, watercolor, award winnin
a bioluminescent jellyfish over futuristic cityscape, deviantart, cinematic lighting, vibrant
hyperrealism painting, underwater sea sponges, tending on deviantart, cinematic lighting, god rays, pastel palette, pastelwave
impressionism painting by Bob Ross, volcano, lava, trees, boulders, obsidian, trending on artstation
realism landscape oil painting, rusty dilapitated metal buildings, foggy ground, graffiti on buildings, vibrant
magical realism painting of a lighting storm over the ocean by Ivan Aivazovsky dark night
naturalism painting of vineyard, vibrant palette, bright sky, clouds
landscape painting of a field of blue roses, a lone greenhouse in the field
realism painting of a pack of wolves in a snowy field, snowing, twilight skies, aurora borealis, by Albert Bierstadt
landscape painting of grassy field with many Pikachu by Albert Bierstadt, afternoon skies, vibrant, trees
futuristic cyber city slums landscape by Liam Wong, highly detailed matte painting
beautiful cyberpunk portrait by Liam Wong and Ross Tran, artstation, dark, contrasting, noir
a ruined cathedral in a field of flowers by Dan Mumford, digital art, gothic art
a ruined building on a field of flowers on a stormy day by Wayne Barlowe and Hans Bellmer, turbulent clouds, concept art, cinematic lighting
a glass tower on a field of flowers landscape by Mike Winkelmann and Beeple, surrealism, futurism
a singular glass tower in a field of flowers landscape by John Atkinson Grimshaw, highly detailed 3d matte painting
massive subterranean galaxy in sprawling ancient ruins of atlantis, a dramatic matte painting by Tyler Edlin, trending on artstation, vivid and vibrant
a stunning painting of a mystical glass tower in the town square by John Atkinson Grimshaw, highly detailed matte painting, nightscape
an ominous painting of the Eiffel tower by Zdzisław Beksiński
a beautiful portrait of mecha statue of liberty by  James Jean and Ross Tran
a magic realism painting by Gediminas Pranckevicius depicting an abandoned building in a field of flowers landscape, vibrant, cinematic lighting
watercolour image of hooded figure standing in small boat collecting orbs of light from a river
a majestic glass tower in a serene oceanic landscape illustrated by john atkinson grimshaw and thomas kinkade, digital art, magic realism
portrait of a stunning woman in los angeles by Peter Mohrbacher and Ross Tran, retrowave noir
sinister abandoned factory building in a desolate landscape by Laurie Lipton and Patrick Woodroffe, withered roses, dramatic scenery, synthwave
a painting of a person standing in front of a house, a matte painting by chris cold, cgsociety, neo-romanticism, matte painting, atmospheric, volumetric lighting
a beautiful portrait of a vine blossom fairy by Ross Tran and Leonardo da Vinci, rossdraws, highly detailed digital painting
a contemporary mansion in a field of flowers landscape by Ross Tran and Caspar David Friedrich, oil on canvas, dramatic scenery, moody palette
a beautiful portrait of a woman by Ross Tran and Leonardo da Vinci, rossdraws, artstation, piercing gaze
robotic android on a balcony pondering the baroque city, artstation, oil and canvas, nightscape, at night
portrait of a brooding magic investigator woman, by Tom Bagshaw and Sophie Anderson and Antoine Blanchard, moody palette, oil on canvas, neon light illumination, noir
aesthetic masterpiece vaporwave woman portrait painting by Antoine Blanchard and Vincent Callebaut
architectural sci-fi brutalist monument, nightscape, surrealism, oil on canvas, award winning, contest winner, by John Atkinson Grimshaw and Thomas Kinkade
steampunk brutalist structural steampunk victorian monument in a field of flowers, deviantart, oil on canvas, moody palette, cinematic lighting
a stunning painting of vintage truck parked on a mountain overlook, vaporwave palette, matte painting, moody palette, deviantart
giant blooming flower on the alien planet topology, render, by Beeple and Gilbert Williams, magical realism, artstation
warm cottage on a black sand beach, digital art, surrealism, dark ominous lonely, oil on canvas, textured painting, american romanticism
cold concrete brutalist spomenik monument in the grand canyon, by john harris and tyler edlin and caspar david friedrich, graphic novel style
portrait of mysterious handsome noir spacetime detective by Dante Gabriel Rossetti and Edward Moran and Charlie Bowater, magical realism, cgsociety, artstation
a vast planetary sci-fi city by Ansel Adams and Bernardo Bellotto, oil on canvas, artstation, dramatic scenery, masterpiece, aesthetic
a stunning profile portrait of art deco empress, by Evelyn De Morgan and Ross Tran, rossdraws, fresco, piercing gaze, black hair
a portrait of a character in a scenic environment by John Atkinson Grimshaw by RHADS and Gilbert Williams, artstation
movie still from a dark city noir thriller sci-fi movie by Monet, vibrant, artstation, HBO, warner brothers
portrait of aesthetic angelic vaporwave noir woman, by John Watkiss, by John Atkinson Grimshaw, soft lighting, deviantart, contrasting, dark, character concept illustration
an aesthetic closeup portrait of a noir woman with red hair painted by John Atkinson Grimshaw and Antoine Blanchard, autumn palette, artstation
a lonely ship adrift in the endless ocean of time, digital art, distant birds, moody palette
descent into infinite madness, surrealism, oil on canvas, texture painting, statement piece, artstation, dark ominous scene, digital art
clockwork wizard tower in the woods, artstation, highly detailed, cinematic lighting, watercolor, masterpiece, aesthetic
cat sitting on the street, well composed cityscape painting, oil on canvas, watercolor, highly detailed impressionist painting
synthwave 1980 sports car on an Arctic landscape, artstation, by Jordan Grimmer and RHADS and Gilbert Williams
aesthetic iridescent crumbling cottage on a cliff by the sea, oil on canvas, artstation, soft render
aesthetic warrior sorceress in scale armor portrait by Raymond Swanland, cyberpunk, sci-fi cybernetic implants
Lilith, ada Lovelace portrait, Deadpool, gothic, artstation, oil on canvas
aesthetic angelic redhead woman stone portrait by Anna Dittmann and Harumi Hironaka and Filip Hodas, flowers, magical, artsy
aesthetic stunning portrait of a woman on the roof watching the sunset by Daniel F. Gerhartz and Liam Wong, nocturne, noir aesthetic
stunning portrait of a man at a piano by Daniel F. Gerhartz and Liam Wong, nocturne, noir aesthetic, neon lights, dark, contrast
stunning portrait of a woman in an office by Daniel F. Gerhartz and Liam Wong, nocturne, noir aesthetic
stunning portrait of a woman walking on the magnificent mile by Wadim Kashin and Liam Wong and Antoine Blanchard, concept art, aesthetic, masterpiece
nocturne impressionist noir of a mysterious detective on the streets by Antoine Blanchard, cityscape, monochromatic, rain reflections, octane soft render, vintage detective
planetary megacity sci-fi city by Hugh Ferriss, masterpiece, dystopian, architectural structural design, contrasting
a stunning portrait of samurai woman by Evelyn De Morgan and Ross Tran, rossdraws, fresco
stunning renaissance fresco portrait, masterpiece, artstation, by Sandro Botticelli, by Sofonisba Anguissola
a portrait of kyrickyoung, oil on canvas, artstation, by J. C. Leyendecker and Edmund Blair Leighton and Charlie Bowater, octane render, soft lighting, contrasting
stunning futuristic sci-fi woman character concept digital art by Leonardo da Vinci and WLOP, surrealism, character illustration, fresco cracked paint, highly detailed sci-fi illustration, soft render
abandoned crumbling shopping mall overgrown by flowers and vines by Tyler Edlin and Jean Delville and John Aktinson Grimshaw and Hubert Robert, german romanticism style, oil on canvas, cinematic lighting, vibrant, hdr, concept art, rain reflections, juxtaposition
the grand canyon by Tyler Edlin and Jean Delville and John Aktinson Grimshaw and Hubert Robert, german romanticism style, oil on canvas, cinematic lighting, vibrant, concept art, gothic, contrasting
chicago magnificent mile overgrown by vines and flowers by Tyler Edlin and Jean Delville and John Aktinson Grimshaw and Hubert Robert, german romanticism style, oil on canvas, cinematic lighting, vibrant, concept art, gothic, contrasting, nightscape
woman ponders the rocky mountains by Jordan Grimmer and RHADS and Gilbert William
majestic frost covered Bison closeup by Jordan Grimmer and RHADS and Gilbert Williams and albert bierstadt, hdr, 4k, snow fog, octane, soft render
Chicago by Jordan Grimmer and RHADS and Gilbert Williams
the rocky mountains by Jordan Grimmer and RHADS
the streets of an abandoned crumbling city overgrown by flowers and vines by Jean Delville and John Aktinson Grimshaw and Hubert Robert, german romanticism style, oil on canvas, cinematic lighting, vibrant, hdr, concept art
atompunk wizard tower in a field of flowers landscape by John Harris and RHADS and Gilbert Williams, highly detailed matte painting, hdr, concept art
astronaut standing in a scenic field of flowers at sunset, synthwave, octane, soft render, artstation
a beautiful closeup portrait of a orange blossom fairy by Ross Tran and Leonardo da Vinci, rossdraws, highly detailed, digital painting, digital art
magnificent fantasy watch-tower inside ruby-crystal in an undersea subterranean landscape, highly detailed, intricate, octane, well lit, waterfalls
imposing stark brutalist cyberpunk monument Spomenik, nightscape, at night a vast planetary sci-fi city by Ansel Adams and Bernardo Bellotto, oil on canvas, award-winning, textured, moody palette
stunning portrait by Charlie Bowater and Monia Merlo, character concept, cgsociety, aviator sunglasses, synthwave, dramatic scenery
stunning beautiful astronaut woman portrait by yildiray cinar and mike allred and charlie bowater, illustration, magical realism, cgsociety, cosmic nebula
a flower galaxy by beeple and gilbert williams, cosmic nebula, digital painting, surrealism
character design samurai cowgirl by Ross Tran and Leonardo da Vinci, rossdraws, octane, highly detailed illustration, character concept design
woman relaxing on the vaporwave beach, dramatic clouds, beautiful day, impasto
magnificent fantasy-watch-tower inside amethyst-crystal in an undersea subterranean landscape, highly detailed, intricate, octane, well lit, waterfalls
stunning portrait illustration of a viking warrior standing on the shore, dramatic scenery, detailed illustration, highly detailed, cyberpunk, vaporwave
stunning portrait of a woman in the ocean, cosmic nebula, beautiful eyes, flowing hair, magical realism, by Ross tran and Charlie Bowater and ivan aivazovsky, ethereal, beautiful, stunning, elegant, peaceful
dark sorceress by scott kolins and charlie bowater, illustration, magical realism, cgsociety, artstation
woman pondering the night, planets visible in the night sky, moon reflecting on the surface of the water, digital art, contest winner, digital painting
an abandoned building under the sea by RHADS and Ivan Aivazovsky, dark realism, digital painting
electronic monolith in the center of a lost village in the mountains, fantasy art, digital art, contest winner, highly detailed
beautiful grand megalopolis designed by Tyler Edlin and Ivan Aivazovsky, digital art, illustration
god in the machine by Jean Delvillle and Peter Mohrbacher, digital art, magical realism, chracter concept
car on an overlook of the vaporwave sunset, still calm sea, vaporwave
glass lighthouse on the ocean by John Atkinson Grimshaw and Tyler Edlin, magical realism, digital painting, trending on artstation, cinematic
painting of an ornate bonsai by monet and john harris and Mikko Lagerstedt, magical realism painting, impressionism, cosmic nebula, holographic
portrait of mysterious handsome noir detective in the ruined spacetime city by Dante Gabriel Rossetti and Edward Moran and Charlie Bowater, magical realism, cgsociety, artstation
portrait of an astronaut in flowerfield landscape, dramatic scenery, magical realism, by Tyler edlin and John Harris
beautiful mysterious gunslinger by Charlie Bowater and Monia Merlo, magical realism, fantasy portrait, character concept
a handsome noir detective woman portrait by Charlie Bowater and Monia Merlo, magical realism, fantasy portrait, character concept, retro futurism
city street at night by Charlie Bowater and William Oxer, retrowave noir
beautiful painting of a cyborg parrot by charlie bowater and ross tran
stunning retro glamor portrait of a woman by Peter mohrbacher and Charlie bowater, cyberpunk, retrowave, highly deatiled painting, octane soft render
portrait of the smoky ash singularity by Tyler Edlin and Peter Mohrbacher and Charlie Bowater
portrait of 80's synthwave sorceress in the cyberpunk cathedral by William Oxer and Charlie Bowater, magical realism, noir aesthetic
stunning portrait of a priestess by Charlie Bowater and WLOP, elegant cyberpunk
stunning religious portrait of elegant cyberpunk priestess by Charlie Bowater and Yigal Ozeri and Ross Tran
stunning religious portrait of an industrial singularity by Peter mohrbacher and Yigal Ozeri, noir aesthetic, retrowave, magical realism
retrowave noir cyberpunk cyborg woman portrait by monia merlo and ross tran and steve hanks, character concept, cgsociety, artstation
unoccupied glitchcore house by Harry Kingsley and Gregory Crewdson, synthwave
retrowave reflecting pool by Harry Kingsley and Gregory Crewdson, magical realism painting
inside unoccupied retrowave office by Harry Kingsley and Gregory Crewdson
a beautiful painting of a brutalist shinto building in a field of flowers by Caspar David Friedrich and Ivan Aivazovsky, german romanticism, contest-winner, beautiful painting, oil on canvas, digital art
"""

builtin_prompts = textwrap.dedent(builtin_prompts.strip())

In [ ]:
# https://huggingface.co/settings/tokens
# Add auth token here

token = ""

In [ ]:
# Initialize the Model

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, 
                                               revision="fp16", 
                                               torch_dtype=torch.float16, 
                                               use_auth_token=token).to("cuda")

# uncomment for SD2

# from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

# model_id = "stabilityai/stable-diffusion-2"
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# pipe = StableDiffusionPipeline.from_pretrained(model_id, 
#                                                revision="fp16", 
#                                                scheduler=scheduler,
#                                                torch_dtype=torch.float16, 
#                                                use_auth_token=token).to("cuda")
    

# 1. How To Train Your Parrot

Prompt Parrot will train on the prompts provided for the number of epochs specified. The more epochs, the longer training will take. But the results will be more accurate to your prompts. If you train too long the Parrot will overfit and produce your prompts exactly. Prompt Parrot handles prompt diversity pretty well. In fact, diverse prompts generally produce better results. If a word or phrase is repeated too much the parrot will fixate on it. This may be good or bad depending on how often you want the modifier to appear in your prompt generations.

Ideal `training loss`, in my experience, is in range 0.2 - 0.3. Start with 5 epoch steps. If training loss is too high lower the steps. But if it's too low, raise the steps. 

In [ ]:
# Parrot Training Settings

mode = "Train New Parrot" #options: "Train New Parrot", "Load Existing Parrot"
model_name = "prompt_parrot"
num_train_epochs=5

# Prompts Settings
prompts_source = "Use Kyrick Prompts" #options: "Use Kyrick Prompts", "Load Text File"
prompts_file_path = "/content/prompts.txt" #@param{type:"string"}

####Legacy Training Mode
# Legacy mode is the old 2.0 training method. It generates less coherent prompts but more creative prompts. It can be fun to use! 
# The results can get pretty crazy. 
# `Set num_train_epochs to at least 50`
classic_choas_mode = False
num_prompt_shuffles = 4

In [ ]:
# Train

saved_model_path = CACHE_PATH / model_name
saved_prompt_txt_path = saved_model_path / "prompts.txt"

tokenizer = AutoTokenizer.from_pretrained("distilgpt2", cache_dir=CACHE_PATH)
tokenizer.add_special_tokens(special_tokens_dict)


mode_map = {"Train New Parrot": "train_model", 
            "Load Existing Parrot": "load_model"}

prompt_map = {"Use Kyrick Prompts": "kyrick_prompts", 
              "Load Text File": "load_prompts_txt"}

mode = mode_map[mode]
prompts_source = prompt_map[prompts_source]

# load all prompts!
all_prompts = []
if mode == "train_model":
    if prompts_source == "kyrick_prompts":
        all_prompts = builtin_prompts.splitlines()
    else:
        with open(prompts_file_path) as infile:
            all_prompts = infile.read().strip().splitlines()
else:
    with open(saved_prompt_txt_path, "r") as infile:
        all_prompts = infile.read().strip().splitlines()

# if no prompts then what we do?!
if not all_prompts:
    raise UserWarning(f"Read 0 prompts from {prompts_file_path}. Please provide prompts in a line separated file called ")


# train or load model
model = None
if mode == "load_model":
    model = AutoModelForCausalLM.from_pretrained(str(saved_model_path))
    print("Model loaded from drive. Training skipped.")
else:
    # create save location
    os.makedirs(str(saved_model_path), exist_ok=True)

    # save prompts used for training
    with open(saved_prompt_txt_path, "w+") as outfile:
        for prompt in all_prompts:
            outfile.write(prompt + "\n")

    # load up the dataset
    prompts_dataset = datasets.Dataset.from_dict({"prompts": all_prompts})
    if classic_choas_mode:
        tokenized_dataset = tokenize_group(
            tokenizer, prompts_dataset, shuffles=num_prompt_shuffles
        )
        if not len(tokenized_dataset):
            # if the batch load fails due to too few prompts, fall back to line by line
            tokenized_dataset = tokenize_line_by_line(tokenizer, prompts_dataset)
    else:
        tokenized_dataset = tokenize_line_by_line(tokenizer, prompts_dataset)

    # train the model
    training_args = TrainingArguments(
        output_dir='./output',
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=1,
        prediction_loss_only=True,
        logging_steps=100,
        save_steps=0,
        seed=random.randint(0, 2**32-1),
    )
    model = AutoModelForCausalLM.from_pretrained("distilgpt2", cache_dir=CACHE_PATH)
    model.resize_token_embeddings(len(tokenizer))
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    trainer.train()

    model.save_pretrained(str(saved_model_path))


In [ ]:
#Generate prompts! Settings

#@markdown _A prompt for my prompts?!?_ **PREPOSTEROUS!!** 
#@markdown
#@markdown GPT-2 requires a prompt string to start a sentence. Best results will resemble part of your prompts. But sometimes original starting strings can have unexpected awesome results!
#@markdown
#@markdown Three prompt modes:
#@markdown * `Starting_Words`: All prompts generated with starting words selected from the start of your prompts. Configure with `num_start_words`
#@markdown * `Promptless`: Uses the start token to generate prompts. Each prompt will start with a different word (usually from your prompts. but it can be suprising!)
#@markdown * `Manual_Prompt`: Input a `manual_prompt` below!
style = "Promptless" #options: "Starting_Words", "Promptless", "Manual_Prompt"
manual_prompt="an imposing dark mysterious tower by Monet"
num_start_words = 2

num_prompts_to_generate=10
#Maximum and min length of the generated prompts. Will cut off mid word. This is expected behavior
max_prompt_length=50 
min_prompt_length=30
#`temperature`: Default: 1.2. Turning up will inject more chaos.
temperature=1.2
#`top_k`: Default 70. The number of top tokens returned by the AI. Will be randomly selected for generation.
top_k=70 
# `top_p`: Default 0.9. The total percent to consider from the `top_k` returned tokens. 
# For more information refer to this guide! https://docs.cohere.ai/token-picking/
top_p=0.9

In [ ]:
#Generate prompts!

if style == "Starting_Words":
    prompt_starts = list(set([" ".join(p.split()[0:num_start_words]).replace(",", "") for p in all_prompts if len(p.split()) > 1]))
    prompt = random.choice(prompt_starts)
elif style == "Promptless":
    prompt = start_token
else:
    if not manual_prompt:
        raise UserWarning("manual_prompt must be at least 1 letter")
    prompt = manual_prompt
    
encoded_prompt = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_length=max_prompt_length,
    min_length=min_prompt_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    do_sample=True,
    num_return_sequences=num_prompts_to_generate,
    pad_token_id=tokenizer.pad_token_id # gets rid of warning
    )

tokenized_start_token = tokenizer.encode(start_token)

prompt_overrides = ""
prompts_to_run = []
for generated_sequence in output_sequences:
    if classic_choas_mode:
        # all the tokens! All the time! 
        tokens = list(generated_sequence)
    else:
        # precision is a virtue
        tokens = []
        for i, s in enumerate(generated_sequence):
            if not classic_choas_mode and s in tokenized_start_token and i != 0:
                if len(tokens) >= min_prompt_length:
                    break
            tokens.append(s)

    text = tokenizer.decode(tokens, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    text = text.strip().replace("\n", " ").replace("/", ",") # / remove slash. It causes problems in namings
    prompts_to_run.append(Prompt(base_filename="PromptParrot", prompt=text))
    print(text) # / remove slash. It causes problems in namings

In [ ]:
# OPTIONAL: copy a subset of prompts into this block quote. Edit them if you'd like. It will override all prompts and only run these!
# example:
# prompt_overrides = """\
# a beautiful painting of a lamb in a pastoral landscape
# a twisted spire on an alien landscape by John Harris and Ross Tran
# """
prompt_overrides = """\
"""

In [ ]:
# Do The Run
prompts_ = (p.strip() for p in prompt_overrides.splitlines() if p.strip())
overrides = [Prompt(base_filename="PromptParrot", prompt=p) for p in prompts_]    

# Run Settings
# set seed to -1 for random seed
batch_name = "PromptParrot" #@param {type: "string"}
# batch_size is images per prompt
batch_size = 6 #@param {type: "integer"}
manual_seed = -1 #@param {type:"integer"}
steps = 50  #@param {type:"integer"}
cfg_scale = 9.0  #@param {type:"number"}
scheduler = "k_lms" #@param ['k_lms', 'ddim', 'pndm']

# set to 0 to turn off. Number of retries per NSFW result (will try with different seed)
NSFW_retries = 1 #@param {type:"integer"}

# ##Image Settings
image_ext = "png" #@param ["jpeg", "png"]
width = 512  #@param {type: "integer"}
height = 512  #@param {type: "integer"}

# turn on/off dispalay of each image
display_single_images = False #@param {type:"boolean"}

# ##Grid Settings
make_batch_grid = True #@param {type:"boolean"}
grid_cols = -1 #@param {type:"integer"}
max_images_per_grid = 6 #@param {type:"integer"}
grid_padding = 5 #@param {type:"integer"}
# color name (`black`, `white`, `pink`, etc.) or hex code in format `#00004c'`
grid_bg_color = "black" #@param {type: "string"}

batch_settings = BatchSettings(
    batch_size=batch_size,
    batch_name=batch_name,
    base_path=IMAGES_OUT,
    steps=steps,
    scheduler=scheduler,
    cfg_scale=cfg_scale,
    seed=manual_seed,
    NSFW_retry=NSFW_retries,
    display_individual_images=display_single_images,
    image_ext=image_ext,
    image_w=width,
    image_h=height,
    make_grid=make_batch_grid,
    grid_cols=grid_cols,
    grid_max_images=max_images_per_grid,
    grid_padding=grid_padding,
    grid_bg_color=grid_bg_color
)

# from diffusers import LMSDiscreteScheduler, DDIMScheduler, PNDMScheduler
# lms = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
# pipe.scheduler = lms
if model_id != "CompVis/stable-diffusion-v1-4":
    set_scheduler(pipe, batch_settings.scheduler)
if overrides:
    run_prompts(pipe, overrides, batch_settings)
else:
    run_prompts(pipe, prompts_to_run, batch_settings)

